![image](https://github.com/ecastillot/SeisMonitor/blob/master/docs/figures/seismonitor.PNG?raw=true)

**This code is necessary on colab to install SeisMonitor.** 

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install SeisMonitor
    !pip install git+https://github.com/ecastillot/EQTransformer.git@master
    !pip install git+https://github.com/wayneweiqiang/GaMMA.git

**Please restart the kernel. It's mandatory to get everything up and running.**

# Data

## Seismic data

There are two main requirements:
- **Downloads folder**. Your data must have the next format (e.g. /downloads_folder/CA06/GS.CA06.00.HHZ__20190902T000000Z__20190903T000000Z.mseed). If your mseed files have different name format you just need to change their structure.
- **Stations folder**. It must be prepared with two files: inv.xml and stations.json. These files contain the metadata. 
You could have an [Inventory](https://docs.obspy.org/packages/autogen/obspy.core.inventory.inventory.Inventory.html) Obspy object, and feel free to use SeisMonitor.monitor.downloader.utils.get_inv_and_json to obtain inv.xml and stations.json information.

Those requirements could be easily obtained following the steps mentioned in *1.downloader*: [![1.downloader](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ecastillot/SeisMonitor/blob/master/examples/1.downloader.ipynb). 

Keep in mind if you have your data in another format, you could use LocalClient (specified en 1.downloader) to load your client, create a provider, and after to use MseedDownloader.


To save time, we will download data already prepared for you. 

In [ ]:
import os
import zipfile
from SeisMonitor.utils4examples import clone_seismonitor_data

monitor_path = os.path.join(os.path.dirname(os.getcwd()),"6m")

clone_seismonitor_data(monitor_path ,branch="dataset")
with zipfile.ZipFile(os.path.join(monitor_path ,"downloads.zip"), 'r') as zip_ref:
    zip_ref.extractall(monitor_path )

downloads = os.path.join(monitor_path ,"downloads")
stations = os.path.join(monitor_path ,"stations")

print("Dataset dir: ",monitor_path )
print("Important folders in your dataset",["downloads","stations"])
print("#"*100)
print("downloads dir: ",downloads)
print("\t",os.listdir(downloads))
print("stations dir: ", stations)
print("\t",os.listdir(stations))

Feel free to take a look at the example data. Please go to the downloads and stations folder to see what should be the format

## Picking Models

We will use pre-trained Deep Learning models to pick seismic phases in your data. SeisMonitor provides you two models: 
- [EQTransformer](https://github.com/smousavi05/EQTransformer/tree/master)
- [PhaseNet](https://github.com/AI4EPS/PhaseNet)

Cloning picking models

In [ ]:
from SeisMonitor.utils4examples import clone_seismonitor_data
models = os.path.join(os.path.dirname(os.getcwd()),"picking_models")
clone_seismonitor_data(models,branch="models")

eqt_model = os.path.join(models,"EQTransformer_models","EqT_model.h5")
pnet_model = os.path.join(models,"PhaseNet_models","190703-214543")

print("Models dir: ",models)
print("Important folders in your models",os.listdir(models))

# Picking with EQTransformer

In [ ]:
import os
from SeisMonitor.monitor.picker.ai import EQTransformer,EQTransformerObj
from SeisMonitor.monitor.picker import utils as piut

We create the object: EQTransformerObj, which contains the EQTransformer parameters.

In [ ]:
eqtobj = EQTransformerObj(model_path=eqt_model,
            n_processor = 6,
            overlap = 0.3,
            detection_threshold =0.1,
            P_threshold = 0.01,
            S_threshold = 0.01,
            batch_size = 20,
            number_of_plots = 0,
            plot_mode = 1 ) 

- EQTransformer is instanced with EQTransformerObj.
- We use **pick** method to pick seismic phases. The input parameters are:
    - Folder where is downloaded the data,
    - Folder where is downloaded the metadata 
    - Output folder from EQTransformer
- We use eqt_picks_2_seismonitor_fmt to export seismic phases in specific format util for the rest of the workflow.

In [ ]:
out_dir = os.path.join(monitor_path ,"picks","eqt")
result = os.path.join(monitor_path ,"picks","eqt","seismonitor_picks.csv")

eqt = EQTransformer(eqtobj)
eqt.pick(downloads,stations,out_dir)
piut.eqt_picks_2_seismonitor_fmt(out_dir,downloads,result)

# Picking with PhaseNet

In [ ]:
import os
from SeisMonitor.monitor.picker.ai import PhaseNet,PhaseNetObj
from SeisMonitor.monitor.picker import utils as piut

To use PhaseNet, please use the next forked repository: 

[PhaseNet](https://github.com/ecastillot/PhaseNet.git)

In [ ]:
pnet_path = os.path.join(monitor_path ,"PhaseNet")
piut.clone_aipicker("PhaseNet",pnet_path)

We create the object: PhaseNetObj, which contains the PhaseNet parameters.

In [ ]:
pnetobj = PhaseNetObj(pnet_path=pnet_path,
            model_path=pnet_model,
            P_threshold=0.7, S_threshold=0.6,
            batch_size=100
            ) 

- PhaseNet is instanced with PhaseNetObj.
- We use **pick** method to pick seismic phases. The input parameters are:
    - Folder where is downloaded the data,
    - Folder where is downloaded the metadata 
    - Output folder from PhaseNet

In [ ]:
out_dir = os.path.join(monitor_path,"picks","pnet")
result = os.path.join(monitor_path,"picks","pnet","seismonitor_picks.csv")

pnet = PhaseNet(pnetobj)
pnet.pick(downloads,stations,out_dir)